# **Limpiando nuestros datos 🧹**

En este primer notebook vamos a cargar nuestros datos y pasar a realizar la limpieza de los mismos, tratar filas incompletas y datos duplicados será una de las tantas tareas que realicemos asi que manos a la obra 🙌

Vamos a cargar nuestros datos y hacer una previsualizacion sencilla

In [1]:
# Importamos las librerías necesarias
import numpy as np  
import pandas as pd

df_headstroke = pd.read_csv('dataset/healthcare-dataset-stroke-data.csv')

# Visualizamos las primeras filas del DataFrame para entender su estructura
df_headstroke.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


### Eliminación de columnas irrelevantes
La columna `id` contiene identificadores únicos para cada paciente. Dado que estos números no guardan relación con el riesgo clínico de sufrir un infarto, procederemos a eliminarla para evitar que el modelo encuentre patrones ruidosos o irrelevantes.

In [2]:
# Eliminamos la columna 'id' de forma permanente
df_headstroke.drop(columns=['id'], inplace=True)

# Verificamos que la columna ha sido eliminada correctamente
df_headstroke.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


### Conteo de filas y columnas

Vamos a contar la cantidad de filas y columnas de nuestro dataset para ello haremos el siguiente codigo de python

In [3]:
print("La cantidad de filas de nuestro dataset es {} y su cantidad de columnas es {} ".format(str(df_headstroke.shape[0])
                                                                                              ,str(df_headstroke.shape[1])))

La cantidad de filas de nuestro dataset es 5110 y su cantidad de columnas es 11 


### Diagnóstico de Calidad de Datos

Un paso fundamental es identificar la presencia de valores nulos y entender la naturaleza de nuestras variables. En contextos médicos, un valor faltante puede ser tan informativo como uno presente.

In [4]:
# Obtenemos un resumen de los tipos de datos y valores no nulos
print("--- Información General del Dataset ---")
df_headstroke.info()

# Contabilizamos los valores nulos por columna
print("\n--- Conteo de Valores Nulos ---")
print(df_headstroke.isnull().sum())

--- Información General del Dataset ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 439.3+ KB

--- Conteo de Valores Nulos ---
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_ty

### Imputación de la variable 'bmi'
Detectamos 201 valores faltantes en `bmi`. Dado que representan menos del 4% del total, utilizaremos la **imputación por mediana**. Elegimos la mediana en lugar de la media porque el IMC suele tener valores extremos (pacientes con obesidad severa) que podrían sesgar el promedio.

In [5]:
# Rellenamos los 201 valores nulos con la mediana de la columna
bmi_median = df_headstroke['bmi'].median()
df_headstroke['bmi'] = df_headstroke['bmi'].fillna(bmi_median)

# Confirmamos que el conteo de nulos sea ahora 0
print(f"Nulos restantes en BMI: {df_headstroke['bmi'].isnull().sum()}")

Nulos restantes en BMI: 0


### Limpieza de consistencia categórica
Revisamos los valores únicos de las variables cualitativas para asegurar que no haya errores de digitación o categorías redundantes.

In [6]:
# Listado de columnas categóricas para revisar
categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

for col in categorical_cols:
    print(f"Categorías en {col}: {df_headstroke[col].unique()}")

# Nota especial: Si en 'gender' aparece 'Other', debemos evaluar si es un error o un dato válido.
# En este dataset suele haber 1 solo registro 'Other', el cual podemos eliminar para simplificar.
df_headstroke = df_headstroke[df_headstroke['gender'] != 'Other']

Categorías en gender: ['Male' 'Female' 'Other']
Categorías en ever_married: ['Yes' 'No']
Categorías en work_type: ['Private' 'Self-employed' 'Govt_job' 'children' 'Never_worked']
Categorías en Residence_type: ['Urban' 'Rural']
Categorías en smoking_status: ['formerly smoked' 'never smoked' 'smokes' 'Unknown']


### Análisis de categorías especiales ('Other' y 'Unknown')
Dentro de nuestras variables categóricas, hemos detectado dos casos que requieren atención:
1. **Gender:** Presencia de una categoría minoritaria llamada 'Other'.
2. **Smoking Status:** Presencia de la etiqueta 'Unknown'.

Vamos a cuantificar cuántos registros representan para decidir la mejor estrategia de tratamiento.

In [7]:
# Contamos la frecuencia de 'Other' en gender
count_other = (df_headstroke['gender'] == 'Other').sum()

# Contamos la frecuencia de 'Unknown' en smoking_status
count_unknown = (df_headstroke['smoking_status'] == 'Unknown').sum()
perc_unknown = (count_unknown / len(df_headstroke)) * 100

print(f"Registros con género 'Other': {count_other}")
print(f"Registros con estado de fumador 'Unknown': {count_unknown} ({perc_unknown:.2f}%)")

Registros con género 'Other': 0
Registros con estado de fumador 'Unknown': 1544 (30.22%)


###  Ejecución del tratamiento
Tras el conteo, aplicamos las siguientes acciones:
* **Eliminar 'Other':** Al ser solo un registro, no permite aprendizaje estadístico y dificulta la binarización.
* **Mantener 'Unknown':** Al ser el 30% del dataset, eliminarlo sería una pérdida masiva de información. Se tratará como una categoría válida ("No reportado").

In [9]:
# 1. Eliminación del registro único 'Other'
df_headstroke = df_headstroke[df_headstroke['gender'] != 'Other']

# 2. Verificación de la limpieza final
print("--- Verificación Final de Categorías ---")
print(f"Géneros restantes: {df_headstroke['gender'].unique()}")
print(f"Dimensiones finales del dataset: {df_headstroke.shape}")

--- Verificación Final de Categorías ---
Géneros restantes: ['Male' 'Female']
Dimensiones finales del dataset: (5109, 11)


### Identificación y tratamiento de registros duplicados
Un error común en la recolección de datos es la duplicidad de registros. Esto puede inflar artificialmente la importancia de ciertos patrones y llevar al sobreajuste (overfitting). Verificaremos si existen filas idénticas en nuestro dataset.

In [11]:
# Contamos el número total de filas duplicadas
duplicados = df_headstroke.duplicated().sum()

print(f"Número de registros duplicados encontrados: {duplicados}")

Número de registros duplicados encontrados: 0


### Exportación del Dataset Limpio
Una vez completada la fase de limpieza técnica (eliminación de IDs, imputación de nulos y ajuste de categorías), procedemos a exportar el DataFrame resultante. Este archivo servirá como punto de partida para el Análisis Exploratorio de Datos (EDA) y el modelado.

In [10]:
# Definimos el nombre del archivo de salida
output_file = 'dataset/healthcare-dataset-stroke-clean.csv'

# Guardamos el dataframe en un nuevo archivo CSV
# index=False evita que se cree una columna adicional con los índices de pandas
df_headstroke.to_csv(output_file, index=False)

print(f"✅ ¡Limpieza completada! El archivo ha sido guardado como: {output_file}")

✅ ¡Limpieza completada! El archivo ha sido guardado como: dataset/healthcare-dataset-stroke-clean.csv
